In [1]:
!pip install transformers
!pip install opendatasets
!pip install pydub
!pip install gdown
!pip install -U sentence-transformers
!pip install resampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=54a9ffe7ff9e24e72bb665a522e0f01204b2f80097bd671023390d40979ef8c7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.8 MB/s eta 0:00:0000:0100:01


In [2]:
import torch
import resampy
from transformers import AutoProcessor, Wav2Vec2Model
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from time import time
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import os
from pydub import AudioSegment
import opendatasets as od
import pandas as pd
import gc
import random
import pickle
import gdown
import json
from nltk import word_tokenize
import string
from zipfile import ZipFile
from torch.nn.functional import cosine_similarity
from tabulate import tabulate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms.functional as F
from PIL import Image
import cv2
from sentence_transformers import SentenceTransformer
from IPython.display import FileLink
from shutil import rmtree
from datasets import load_dataset
from IPython.display import Audio, display
import torchaudio

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
audio_processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
audio_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
vision_model = models.efficientnet_b7(pretrained=True)
vision_model.cuda()
_ = vision_model.eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth
100%|██████████| 255M/255M [00:01<00:00, 213MB/s]  


In [6]:
text_model = SentenceTransformer('sentence-transformers/LaBSE').to(device)

In [7]:
data_path = "/kaggle/input/common-voice-dataset-version-4/data-file/train.tsv"
audio_path = "/kaggle/input/common-voice-dataset-version-4/new-clip"
data_train = pd.read_csv(data_path,comment='#',sep="\t")

In [8]:
len(data_train)

232975

In [14]:
data_train.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accent
0,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664034.mp3,"These data components in turn serve as the ""bu...",2,0,thirties,male,NaN
1,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664035.mp3,The church is unrelated to the Jewish politica...,3,0,thirties,male,NaN
2,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664037.mp3,The following represents architectures which h...,2,0,thirties,male,NaN
3,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664038.mp3,"Additionally, the pulse output can be directed...",2,0,thirties,male,NaN
4,4f29be8fe932d773576dd3df5e111929f4e22242232245...,common_voice_en_19664040.mp3,The two are robbed by a pickpocket who is losi...,3,0,thirties,male,NaN


In [ ]:
data_train['sentence'][4]

In [ ]:
display(Audio(f"{audio_path}/{data_train['path'][4]}"))

In [13]:
audio_file = AudioSegment.from_file(f"{audio_path}/{data_train['path'][4]}")

In [15]:
len(audio_file.get_array_of_samples())

223488

In [20]:
audio_file.frame_rate

48000

In [9]:
# preprocess sentence
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentence = sentence.strip()
    return sentence

In [20]:
preprocess_sentence(data_train['sentence'][10])

'what did you think of that trip'

In [10]:
def get_image_embedding(path):
    image = cv2.imread(path)
    image_tensor = torch.from_numpy(image)
    image_tensor = image_tensor.unsqueeze(0)
    image_tensor = image_tensor.permute(0, 3, 1, 2)
    image_tensor = image_tensor.to(device)
    image_tensor = image_tensor.float()
    with torch.no_grad():
        output = vision_model(image_tensor)
        embedding = output[0].cpu()
        return embedding

In [11]:
start_id = 200000

In [12]:
def pipeline_for_common_voice(dataset, limit, split, valid_ids, part_number, start_idx=0):
    data_v1 = {}
    counter = 0
    j = 0
    max_id = 0
    if start_idx == 0: 
        val_ids_final = valid_ids
    else:
        for i, item in enumerate(valid_ids):
            if item == start_idx:
                j = i
                break
        val_ids_final = valid_ids[(j + 1):]
    for audio_idx in tqdm(val_ids_final):
        if counter == 0:
            print(f'start_idx {audio_idx}')
        if counter % 100 == 0:
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.ipc_collect()
        if counter == limit + 50:
            max_id = audio_idx
            break
        try:
            file_path = f"{audio_path}/{dataset['path'][audio_idx]}"
            inner_data = {}
            inner_data['id'] = start_id + audio_idx
            inner_data['file_path'] = file_path
            inner_data['audio_idx'] = audio_idx

            audio_file = AudioSegment.from_file(file_path)
            samples = np.array(audio_file.get_array_of_samples())
            samples = samples.reshape(-1, audio_file.channels)
            samples = samples / np.max(np.abs(samples))
            samples = samples.squeeze()

            samples = resampy.resample(samples, audio_file.frame_rate, 16000)
            audio = torch.from_numpy(samples)
            inputs = audio_processor(audio, sampling_rate=16000, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = audio_model(**inputs)
            last_hidden_states = outputs.last_hidden_state.squeeze(0)
            embeddings = last_hidden_states.mean(dim=0)
            inner_data['audio_embedding'] = embeddings.cpu()
            inner_data['text'] = preprocess_sentence(dataset['sentence'][audio_idx])
            data_v1[start_id + audio_idx] = inner_data
            counter += 1
        except Exception as e:
            print(e)
            print(dataset['path'][audio_idx])
            continue
    print(f'max id {max_id}')
    data_v2 = {}
    counter = 0
    for audio_idx in tqdm(val_ids_final):
        if counter == limit:
            break

        if start_id + audio_idx not in data_v1.keys():
             continue
                
        if counter % 100 == 0:
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.ipc_collect()

        try:
            file_path = f"{audio_path}/{dataset['path'][audio_idx]}"
            y, sr = librosa.load(file_path)
            spec = librosa.stft(y)
            spec_db = librosa.amplitude_to_db(abs(spec))
            
            plt.clf()
            librosa.display.specshow(spec_db, x_axis='time', y_axis='log')
            plt.xlabel('')
            plt.ylabel('')
            plt.tight_layout()

            saved_path = f'spec.png'
            plt.savefig(saved_path, bbox_inches='tight', pad_inches=0)
            image_embeddings = get_image_embedding(saved_path)
            data_v2[start_id + audio_idx] = data_v1[start_id + audio_idx].copy()
            data_v2[start_id + audio_idx]['image_embedding'] = image_embeddings
            counter += 1

        except Exception as e:
            print(e)
            print(dataset['path'][audio_idx])
            continue
    data_v3 = {}
    counter = 0
    for audio_idx in tqdm(val_ids_final):
        if start_id + audio_idx not in data_v2.keys():
            continue
        if counter == limit:
            break
        try:
            text_embeddings = text_model.encode(data_v2[start_id + audio_idx]['text'])
            data_v3[start_id + audio_idx] = data_v2[start_id + audio_idx].copy()
            data_v3[start_id + audio_idx]['text_embedding'] = torch.tensor(text_embeddings)
            counter += 1
        except Exception as e:
            print(e)
            print(dataset['path'][audio_idx])
            continue
    with open(f'pretrained_embeddings_common_voice_{split}_part_{part_number}.pkl', 'wb') as f:
        pickle.dump(data_v3, f)
    print(f'max id {max_id}')
    return max_id

In [13]:
def find_proper_audios(dataset):
    audio_idxs = []
    for audio_idx in tqdm(range(len(dataset))): 
        file_path = f"{audio_path}/{dataset['path'][audio_idx]}"
        if os.path.exists(file_path):
            audio_idxs.append(audio_idx)
    print(f'len of audio idxs {len(audio_idxs)}')
    return audio_idxs

In [22]:
train_valid_ids = find_proper_audios(data_train)

100%|██████████| 232975/232975 [12:05<00:00, 321.04it/s]

len of audio idxs 178211


In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=1, start_idx=0)

  0%|          | 0/178211 [00:00<?, ?it/s]

start_idx 0


  3%|▎         | 5100/178211 [26:59<15:16:17,  3.15it/s]


max id 18105


  0%|          | 1/178211 [00:01<81:40:06,  1.65s/it]/tmp/ipykernel_28/3664502682.py:77: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
  3%|▎         | 4605/178211 [51:40<37:40:45,  1.28it/s]

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=2, start_idx=max_idx)

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=3, start_idx=max_idx)

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=4, start_idx=max_idx)

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=5, start_idx=max_idx)

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=6, start_idx=max_idx)

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=7, start_idx=max_idx)

  3%|▎         | 5050/147855 [29:29<13:54:08,  2.85it/s]


max id 71054


  0%|          | 1/147855 [00:04<177:53:54,  4.33s/it]/tmp/ipykernel_28/998898637.py:77: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
  0%|          | 179/147855 [02:05<28:40:00,  1.43it/s]

In [23]:
# max_idx = 71054

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=8, start_idx=max_idx)

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=9, start_idx=max_idx)

In [ ]:
# max_idx = pipeline_for_common_voice(data_train, 5000, 'train', train_valid_ids, part_number=10, start_idx=max_idx)

In [14]:
dev_data_path = "/kaggle/input/common-voice-dataset-version-4/data-file/dev.tsv"
data_dev = pd.read_csv(dev_data_path,comment='#',sep="\t")

In [15]:
len(data_dev)

15531

In [16]:
dev_valid_ids = find_proper_audios(data_dev)

100%|██████████| 15531/15531 [00:33<00:00, 457.66it/s]

len of audio idxs 3492


In [ ]:
max_idx = pipeline_for_common_voice(data_dev, 5000, 'validation', dev_valid_ids, part_number=1, start_idx=0)

In [18]:
test_data_path = "/kaggle/input/common-voice-dataset-version-4/data-file/test.tsv"
data_test = pd.read_csv(test_data_path,comment='#',sep="\t")
len(data_test)

15531

In [19]:
test_valid_ids = find_proper_audios(data_test)

100%|██████████| 15531/15531 [00:33<00:00, 461.98it/s]

len of audio idxs 2197


In [ ]:
max_idx = pipeline_for_common_voice(data_test, 5000, 'test', test_valid_ids, part_number=1, start_idx=0)

In [1]:
# mix all files pretrained
import pickle
from IPython.display import FileLink

data = {'train': {}, 'test': {}, 'validation': {}}
for i in range(1, 11):
    with open(f'pretrained_embeddings_common_voice_train_part_{i}.pkl', 'rb') as f:
        data['train'].update(pickle.load(f))
with open(f'pretrained_embeddings_common_voice_test_part_1.pkl', 'rb') as f:
    data['test'].update(pickle.load(f))
with open(f'pretrained_embeddings_common_voice_validation_part_1.pkl', 'rb') as f:
    data['validation'].update(pickle.load(f))

with open(f'pretrained_embeddings_common_voice.pkl', 'wb') as f:
    pickle.dump(data, f)

In [6]:
FileLink('pretrained_embeddings_common_voice.pkl')

/kaggle/working/pretrained_embeddings_common_voice.pkl

In [4]:
with open(f'pretrained_embeddings_common_voice.pkl', 'rb') as f:
    data = pickle.load(f)

In [7]:
print(len(data['train']))
print(len(data['validation']))
print(len(data['test']))

50000
3492
2197


In [ ]:
random_key = random.choice(list(data['train'].keys()))
data['train'][random_key]